# Generating Text with Neural Networks


# Getting the Data

In [1]:
import tensorflow as tf

shakespeare_url = "https://homl.info/shakespeare"  # shortcut URL
filepath = tf.keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

1115394/1115394 [==============================] - 0s 0us/step


In [2]:
print(shakespeare_text[:80]) # not relevant to machine learning but relevant to exploring the data

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.


1. This code runs the first 80 characters in the data set. It has not been shuffled yet and includes punctuations, spaces and 'enter'/line breaks. The colon before the number 80 means only the data entries 0-79 making 80 entries. 

# Preparing the Data

In [3]:
text_vec_layer = tf.keras.layers.TextVectorization(split="character",
                                                   standardize="lower")
text_vec_layer.adapt([shakespeare_text])
encoded = text_vec_layer([shakespeare_text])[0]

In [4]:
print(text_vec_layer([shakespeare_text]))

tf.Tensor([[21  7 10 ... 22 28 12]], shape=(1, 1115394), dtype=int64)


In [5]:
encoded -= 2  # drop tokens 0 (pad) and 1 (unknown), which we will not use
n_tokens = text_vec_layer.vocabulary_size() - 2  # number of distinct chars = 39
dataset_size = len(encoded)  # total number of chars = 1,115,394

In [6]:
print(n_tokens, dataset_size)

39 1115394


In [7]:
def to_dataset(sequence, length, shuffle=False, seed=None, batch_size=32):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda window_ds: window_ds.batch(length + 1))
    if shuffle:
        ds = ds.shuffle(100_000, seed=seed)
    ds = ds.batch(batch_size)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

In [8]:
length = 100
tf.random.set_seed(42)
train_set = to_dataset(encoded[:1_000_000], length=length, shuffle=True,
                       seed=42)
valid_set = to_dataset(encoded[1_000_000:1_060_000], length=length)
test_set = to_dataset(encoded[1_060_000:], length=length)

# Building and Training the Model

In [9]:
tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    "my_shakespeare_model", monitor="val_accuracy", save_best_only=True)
history = model.fit(train_set, validation_data=valid_set, epochs=10,
                    callbacks=[model_ckpt])

Epoch 1/10
  31247/Unknown - 2174s 69ms/step - loss: 1.3902 - accuracy: 0.5744

INFO:tensorflow:Assets written to: my_shakespeare_model\assets


INFO:tensorflow:Assets written to: my_shakespeare_model\assets


31247/31247 [==============================] - 2230s 71ms/step - loss: 1.3902 - accuracy: 0.5744 - val_loss: 1.6044 - val_accuracy: 0.5340
Epoch 2/10
31247/31247 [==============================] - ETA: 0s - loss: 1.2918 - accuracy: 0.5977

INFO:tensorflow:Assets written to: my_shakespeare_model\assets


...

INFO:tensorflow:Assets written to: my_shakespeare_model\assets


31247/31247 [==============================] - 2316s 74ms/step - loss: 1.2440 - accuracy: 0.6082 - val_loss: 1.5486 - val_accuracy: 0.5491
Epoch 10/10
31247/31247 [==============================] - ETA: 0s - loss: 1.2423 - accuracy: 0.6085

INFO:tensorflow:Assets written to: my_shakespeare_model\assets


INFO:tensorflow:Assets written to: my_shakespeare_model\assets


31247/31247 [==============================] - 2281s 73ms/step - loss: 1.2423 - accuracy: 0.6085 - val_loss: 1.5554 - val_accuracy: 0.5492


In [10]:
shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X - 2),  # no <PAD> or <UNK> tokens
    model
])

# Generating Text

In [11]:
y_proba = shakespeare_model.predict(["To be or not to b"])[0, -1]
y_pred = tf.argmax(y_proba)  # choose the most probable character ID
text_vec_layer.get_vocabulary()[y_pred + 2]

1/1 [==============================] - 1s 805ms/step


'e'

In [12]:
log_probas = tf.math.log([[0.5, 0.4, 0.1]])  # probas = 50%, 40%, and 10%
tf.random.set_seed(42)
tf.random.categorical(log_probas, num_samples=8)  # draw 8 samples

<tf.Tensor: shape=(1, 8), dtype=int64, numpy=array([[0, 1, 0, 2, 1, 0, 0, 1]], dtype=int64)>

In [13]:
def next_char(text, temperature=1):
    y_proba = shakespeare_model.predict([text])[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0, 0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

In [14]:
def extend_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [15]:
tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU

In [16]:
print(extend_text("To be or not to be", temperature=0.01))

1/1 [==============================] - 0s 92ms/step
To be or not to be a strange
and so much shall be so with his father


In [17]:
print(extend_text("To be or not to be", temperature=1))

1/1 [==============================] - 0s 67ms/step
To be or not to be broke a grumio!
what's my instruction own thing
a


In [18]:
print(extend_text("To be or not to be", temperature=100))

1/1 [==============================] - 0s 70ms/step
To be or not to bef ,mt'&o3f:ady-$
wh!nse?pws3ert--vgerdjw?c-y-ewxnj
